## Table of contents
* [Authentication](# authentication)
* [Extraction](#extraction)

<img src="reddit.png" style="width: 800px"/>

### 1.Authentication  

PRAW is shortform for **P**ython **R**eddit **A**PI **W**rapper  

**Prerequisites**   
*Python Knowledge*:  
 	You need to know at least a little Python to use PRAW; it’s a Python wrapper after all. PRAW supports Python 2.7, and Python 3.3 to 3.6.
    
*Reddit Knowledge: *   
 	A basic understanding of how reddit.com works is a must. In the event you are not already familiar with Reddit start with their FAQ.  
    
*Reddit Account:*  
A Reddit account is required to access Reddit’s API. Create one at reddit.com. 

*Client ID & Client Secret:  *  
 	These two values are needed to access Reddit’s API as a script application (see Authenticating via OAuth for other application types). If you don’t already have a client ID and client secret, follow Reddit’s First Steps Guide to create them.  
    
*User Agent:*  
A user agent is a unique identifier that helps Reddit determine the source of network requests. To use Reddit’s API, you need a unique and descriptive user agent.
    
[Source](https://praw.readthedocs.io/en/latest/getting_started/quick_start.html)


After creating the credentials (client_id, client_secret, username, password), the next step is to login into reddit API using praw package

In [2]:
import praw
reddit = praw.Reddit(user_agent='Comment Extraction (by /u/USERNAME)',
                     client_id='Mjk1ISLhBg6bXQ', client_secret="K0LPk-78K3Mmhnc2qUuVc_4Dv4Y",
                     username='akhil44', password='$4Akhilesh')

Here, we are creating an authorized reddit instance and not only the read-only instance as authorized instance has less restrictions in terms of data retrieval. The credentials are masked due to privacy and security reasons but you can find the guide to create them in a short time [here]().

### Data extraction
Here in this section, we send the requests to the reddit api and get the headers and body of the comments in 2 separate lists

But we need all the comments including the nested ones. Below code contains another while loop that goes into the comments section and gets the replies as well for those comments.  
All in all, it first goes into the subreddit and then into the comment and gets all the replies and iterates the same process, till it reaches the end of the line in the sub reddit

In [3]:
comm_list = []
header_list = []
i = 0
for submission in reddit.subreddit('cordcutters').hot(limit=2):
    submission.comments.replace_more(limit=None)
    print(i)
    i = i + 1
    comment_queue = submission.comments[:]  # Seed with top-level
    while comment_queue:
        header_list.append(submission.title)
        comment = comment_queue.pop(0)
        comm_list.append(comment.body)
        t = []
        t.extend(comment.replies)
        while t:
            header_list.append(submission.title)
            reply = t.pop(0)
            comm_list.append(reply.body)

0
1


Here, a dataframe has been created by concatenating the headers and comments from the sub reddit

In [5]:
import pandas as pd
df = pd.DataFrame(header_list)
df['comm_list'] = comm_list
df.head()

,0,comm_list
0,"I'm Jason Coyle, CEO of Stadium, the first 24/...",Any plans to expand beyond your Chicago headqu...
1,"I'm Jason Coyle, CEO of Stadium, the first 24/...","We have a sales office in NYC and Det., and we..."
2,"I'm Jason Coyle, CEO of Stadium, the first 24/...","As a college football fan, I really enjoy your..."
3,"I'm Jason Coyle, CEO of Stadium, the first 24/...",Thank you - appreciate that. We have a show o...
4,"I'm Jason Coyle, CEO of Stadium, the first 24/...","In 2021, what do you think the main difference..."


From the raw data, we can see that there are \n tags at the end of every sentence. After some cleaning, we finally export the data to a csv file

In [31]:
df.columns = ['header','comments']
df['comments'] = df['comments'].apply(lambda x : x.replace('\n',''))
df.to_csv('cordcutter_comments1.csv',index = False)

In [33]:
df.loc[673,'comments']

"If you get downto -96 power you'll still get the same sensitivity.  So down to KXLY (probably).  You'll want to raise the antenna as high as possible, so it can see over those edges."